# Compute Pi using `MPI_THREAD_FUNNELED`

***
#### C program

In [1]:
%%file pi-hybrid.c
#include <stdio.h>
#include <omp.h>
#include <mpi.h>

#define N 1000000

int main()
{
  int rank;
  int size;
  double subsum = 0.0;

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);
  omp_set_num_threads(2);
  
  int nthreads = omp_get_num_threads();
  #pragma omp parallel
  {  
    int tid = omp_get_thread_num();
    printf("Thread %d within rank %d started.\n", tid, rank);
    #pragma omp for reduction(+:subsum)
    for(int i = rank; i < N; i += size*nthreads)
      {
        double x = (i+0.5)/N;
        subsum += 4/(1 + x*x);
      }
  }

  double sum;
  MPI_Reduce(&subsum, &sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
  if (rank == 0)
     printf("pi = %.10lf\n", sum/N);

  MPI_Finalize();
  return 0;
}

In [ ]:
!mpicc -fopenmp pi-hybrid.c && mpirun -np 3 --allow-run-as-root a.out

***
#### Fortran program

In [ ]:
%%file pi-hybrid.f90
program hybrid
use mpi

integer, parameter :: N = 1000000
integer ( kind = 4 ) error
integer :: rank, size, nthreads, tid
double precision :: subsum, sum, x
nthreads = 2
subsum = 0.0

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)
call omp_set_num_threads(nthreads)

!$OMP parallel
    tid = omp_get_thread_num()
    print *, "Thread", tid, "within rank", rank, "started."
    !$OMP do reduction(+:subsum)
        do i = rank, N, size*nthreads
            x = (i+0.5)/N
            subsum = subsum + 4/(1 + x*x)
        end do
    !$OMP end do
!$OMP end parallel

call MPI_Reduce(subsum, sum, 1, MPI_DOUBLE_PRECISION, MPI_SUM, 0, MPI_COMM_WORLD, error)    
if (rank == 0) then
    print *, "pi =", sum/N
endif

call MPI_Finalize(error)
end

In [ ]:
!mpif90 -fopenmp pi-hybrid.f90 && mpirun -np 3 --allow-run-as-root a.out